In [120]:
import os
from pathlib import Path as pt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

In [2]:
%matplotlib widget
plt.rc('font', family='serif')

In [148]:
def generateNGaussian(N):

    gaussfn = lambda n: f"A{n}*(np.exp((-1.0/2.0)*(((x-cen{n})/sigma{n})**2)))+"
    _gfn, _args = "", "x, "
    for i in range(int(N)): 
        _gfn += gaussfn(i)
        _args += f"cen{i}, A{i}, sigma{i}, "
    exec(f"gfn_ = lambda {_args.strip()[:-1]}: {_gfn.strip()[:-1]}")
    
    return locals()["gfn_"]

def computeNGaussian(wn, inten, sigma=5):
    
    _args = {}
    N = len(wn)
    gfn = generateNGaussian(N)
    
    i = 0
    for x, y in zip(wn, inten):
        _args[f"cen{i}"] = x
        _args[f"A{i}"] = y
        _args[f"sigma{i}"] = sigma
        i += 1
    
    full_wn = np.linspace(wn.min()-100, wn.max()+100, 5000)
    full_inten = gfn(full_wn, **_args)
    
    return full_wn, full_inten

In [220]:
location = pt("D:\SurfDrive\FELIX_Measurements\hydroxylamine\IR_calculation")
files = list(location.glob("*.txt"))

In [292]:
def exp_plot(filename, ls, ax, ax1, fig):
          
    with open(filename, "r") as f:
        readfile = f.readlines()
    legendName = readfile[0].split(" ")[-1].split("\n")[0]

    wn, inten, wn_anh, inten_anh = np.genfromtxt(filename).T

    #wn_sim, inten_sim = computeNGaussian(wn, inten, 10)
    #ax.plot(wn_sim, inten_sim, "k", label="Harmonic MP2(full)/cc-PVTZ")

    wn_sim_anh, inten_sim_anh = computeNGaussian(wn_anh, inten_anh, 10)
    ax.plot(wn_sim_anh, inten_sim_anh, f"k{ls}", label=legendName)
    ax1.plot(wn_sim_anh, inten_sim_anh, f"k{ls}", label=legendName)


    #ax.set(xlabel="Wavenumber $cm^{-1}$", ylabel="Intensity (km/mol)")
    ax.legend()
    ax1.yaxis.tick_right()
    plt.suptitle(f"Anharm. MP2(full)/cc-PVTZ level")
    
    return ax, ax1

In [293]:
grid = {"hspace":0.05, "wspace":0.07, "width_ratios":[1, 0.5]}
fig, (ax, ax1) = plt.subplots(ncols=2, dpi=100, gridspec_kw=grid)

ls = ("-", "--")
for i, filename in enumerate(files[1:]):
    ax, ax1 = exp_plot(filename, ls[i], ax, ax1, fig)

# Set common labels
fig.text(0.5, 0.01, 'Wavenumber $cm^{-1}$', ha='center', va='center')
fig.text(0.05, 0.5, 'Intensity ($km/mol$)', ha='center', va='center', rotation='vertical')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.05, 0.5, 'Intensity ($km/mol$)')

In [294]:
fig1, (ax2, ax3) = plt.subplots(ncols=2, dpi=100, gridspec_kw=grid)
ax2, ax3 = exp_plot(files[0], "-", ax2, ax3, fig1)
# Set common labels
fig1.text(0.5, 0.01, 'Wavenumber $cm^{-1}$', ha='center', va='center')
fig1.text(0.05, 0.5, 'Intensity ($km/mol$)', ha='center', va='center', rotation='vertical')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.05, 0.5, 'Intensity ($km/mol$)')

In [295]:
fig.savefig(location/f"report/images/NH2OH2+.jpg")
fig.savefig(location/f"report/images/NH2OH2.pdf")

fig1.savefig(location/f"report/images/NH2OH+.jpg")
fig1.savefig(location/f"report/images/NH2OH.pdf")

In [281]:
plt.close("all")